In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Basics of the method

This is an iterative method for approximating the solution at optimizing a function. Say we have the function,

$$ y = w_0 + w_1x_1 + w_2x_2 + \dots + w_kx_k $$

and we make a prediction such as 

$$ \hat{y} = \hat{w_0} + \hat{w_1}x_1 + \hat{w_2}x_2 + \dots + \hat{w_k}x_k $$

This is the regression problem we studied back in Week 3

The method starts with an initial value for each coefficient $\hat{w_i}$ and a prediction $\hat{y}$ is made. 

An *error*, $e$, is computed as $e = \hat{y} - y$ and then each coefficient is updated, for this particular problem, in a new step like:

$$\hat{w_i}[t+1] = \hat{w_i}[t] - l*e*x_i[t]$$

The parameter $l$ is a small chosen constant referred to as *learning factor*. This procedure is repeated for each observation of the training dataset. If at the end of this process the error is still not close enough to zero, we repeat the whole process again, for a number of times or *epochs*.  



In [ ]:
# Let's demo with a small amount of data (x, y) of values 
smalldata = np.array([[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]) 
data = pd.DataFrame(smalldata, columns=['x1', 'y'])
print (data)

In [ ]:
# a Plot is like
sns.regplot('x1', 'y', data=data, fit_reg=False)
plt.xlabel('x')
plt.ylabel('y');

Let's assume that the above 5 points tend to follow a linear trend of the kind $y = w_0 + w_1x$. So let's try to find the coefficient values $w_0, w_1$ that define the line


#### Let's fix the shape of our data

In general,

$$ y = w_0 + w_1x_1 + w_2x_2 + \dots + w_kx_k $$

This equation is normally written as:

$$y = w_0 + \begin{bmatrix}w_1 & w_2 &\dots & w_k\end{bmatrix} \begin{bmatrix}x_1 \\ x_2\\ \dots \\x_k \end{bmatrix} $$

And this can be re-arranged by inserting coefficient $w_0$ inside the matrix $W$ and augmenting feature $X$ with a constant of 1. Like so,

$$y = \begin{bmatrix}w_0 & w_1 & w_2 &\dots & w_k\end{bmatrix} \begin{bmatrix}1 \\ x_1 \\ x_2\\ \dots \\x_k \end{bmatrix} $$

This equation above is for a simple observation. 

The above equation is what most books write as: $y = W^TX$. We prefer to arrange $X$ where each row is an observation and each column is a feature so $X$ looks like a matrix. 

We need to augment our data to have $x_0$ column made of ones for our problem. Let's see how. 

In [ ]:
# adding a column with constant ones
data['x0'] = 1
data

In [ ]:
#Let's change the order of our columns
data = data[['x0', 'x1', 'y']]
data

In [ ]:
# Preparing our data with features and target
X = data.drop(['y'], axis=1)
y = data['y']
X

#### Let's put the method in Python code

In [ ]:
# An initial guess for W, say of zero
W = pd.DataFrame(np.zeros(2).reshape(1, 2), columns = ['w0', 'w1']) # This is stored as a vector 

print ('Initial weights', W)

# A small constant learning factor
l = 0.001

# An arbitrary number of epochs
P = 100

# The gradient descent algorithm
for t in range(P):                   # The number of epochs
    for i in range(5):               # Visit all five points per epoch
        yp = np.dot(W.values, X.iloc[i].values )     # prediction
        e = yp - y[i]                                # error
        W = W - l*e*X.iloc[i].values                 # update of weights
        

In [ ]:
# Final weights
W

### Let's plot what we got


In [ ]:
xtest = np.linspace(0, 8, 100)
ytest = W['w0'].values + W['w1'].values*xtest

sns.regplot('x1', 'y', data=data, fit_reg=True)
sns.lineplot(xtest, ytest )
plt.xlabel('x')
plt.ylabel('y');

Note that the blue line is the one painted by Seaborn using regression as computed in Week 3. The orange line is the one we have learned from Stochastic Gradient Descent method. 

#### Let's plot the error as we advance through the epochs 

In [ ]:
# An initial guess for W, say of zero
W = pd.DataFrame(np.zeros(2).reshape(1, 2), columns = ['w0', 'w1']) # This is stored as a vector 

# A small constant learning factor
l = 0.001

# An arbitrary number of epochs
P = 100

error_history = np.zeros(P)

# The gradient descent algorithm
for t in range(P):                    # The number of epochs
    for i in range(5):                # Visit all five points per epoch
        yp = np.dot(W.values, X.iloc[i].values )     # prediction
        e = yp - y[i]                                # error
        W = W - l*e*X.iloc[i].values                 # update of weights
    error_history[t] = e

In [ ]:
sns.lineplot(np.arange(P), error_history )
plt.ylim(-5,0)
plt.title('Error reduction by epoch with SGD')
plt.xlabel('epochs')
plt.ylabel('error');

Note that keeping the data as a Pandas data frame makes the code a little cumbersome. In next examples we'll keep the computations as numpy arrays